In [2]:
!pip install -q pandas
!pip install dask
!pip install pyarrow
!pip install fastparquet phonetics fuzzywuzzy jellyfish -q
!pip install seaborn -q 
!pip install matplotlib -q 

import pandas as pd
!pip install recordlinkage
import recordlinkage as rl
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import dask.dataframe as dd
import dask
from recordlinkage.preprocessing import clean, phonetic

def merge_tid_lmk(df):
  # df = pd.merge(df, tran[["transactionid", "postcode"]], on = "transactionid")
  # df = pd.merge(df, tran[["postcode"]].reset_index(), on = "transactionid")
  df = pd.merge(df, tran.reset_index(), on = "transactionid")
  df = df.rename(columns = {"postcode":"postcode_tran"})
  # df = pd.merge(df, epc[["lmk", "postcode"]], on = "lmk")
  # df = pd.merge(df, epc[["postcode"]].reset_index(), on = "lmk")
  df = pd.merge(df, epc.reset_index(), on = "lmk")
  df = df.rename(columns = {"postcode":"postcode_epc"})
  return df

def get_matches(df, cols):
  df = df.reset_index()
  df = merge_tid_lmk(df)
  for col in cols:
    df = df[df[col] != 0]
  return df

# get cpu number
import multiprocessing
multiprocessing.cpu_count()

# get RAM
!cat /proc/meminfo

MemTotal:       258379312 kB
MemFree:        66647516 kB
MemAvailable:   79860932 kB
Buffers:            2244 kB
Cached:         14871160 kB
SwapCached:            0 kB
Active:          2280900 kB
Inactive:       187813880 kB
Active(anon):        988 kB
Inactive(anon): 175221204 kB
Active(file):    2279912 kB
Inactive(file): 12592676 kB
Unevictable:          28 kB
Mlocked:              28 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:                16 kB
Writeback:             0 kB
AnonPages:      175217772 kB
Mapped:           496912 kB
Shmem:              1040 kB
KReclaimable:     528320 kB
Slab:             705364 kB
SReclaimable:     528320 kB
SUnreclaim:       177044 kB
KernelStack:       14320 kB
PageTables:       712972 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:    129189656 kB
Committed_AS:   179491380 kB
VmallocTotal:   34359738367 kB
VmallocUsed:       23924 kB
VmallocChunk:          0 kB
Percpu:    

In [3]:
!ls -l ../Census_samples/Whole_ipum/Whole_1861

-rw-r--r-- 1 xiet13 cluster-users 780852725 Nov 19 18:08 ../Census_samples/Whole_ipum/Whole_1861


In [4]:
from dask.distributed import LocalCluster
cluster = LocalCluster(n_workers=8, threads_per_worker=4)

/users/xiet13/.conda/envs/hist_link/lib/python3.8/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 34283 instead
  warnings.warn(


In [5]:
# subSample1851 = dd.read_parquet('../Census_samples/Whole_ipum/Whole_1881').compute() # 17,711,030

# _1851 = dd.read_parquet('../Census_samples/Whole_ipum/Whole_1851').compute()  
# _1861 = dd.read_parquet('../Census_samples/Whole_ipum/Whole_1861').compute()  
# _1881 = dd.read_parquet('../Census_samples/Whole_proc/Whole_1881').compute() 
# _1891 = dd.read_parquet('../Census_samples/Whole_proc/Whole_1891').compute() 
# _1901 = dd.read_parquet('../Census_samples/Whole_proc/Whole_1901').compute() 
_1911 = dd.read_parquet('../Census_samples/Whole_ipum/Whole_1911')
# _1911 = dd.read_parquet('../Census_samples/Whole_ipum/Whole_1911').compute(scheduler = cluster) # need full treatment


In [ ]:
_1911

In [ ]:
features = ['recid', 'pname', 'sname', 'subdist_birth_ID', 'CONPARID_birth',\
                'recid_mom', 'recid_pop', 'recid_sp']

In [15]:
# _1851 = dd.read_parquet('../Census_samples/Whole_proc/Whole_1851')
# _1891 = dd.read_parquet('../Census_samples/Whole_proc/Whole_1891')
# _1901 = dd.read_parquet('../Census_samples/Whole_proc/Whole_1901')
# _1911 = dd.read_parquet('../Census_samples/Whole_ipum/Whole_1911')

# # # print folujns 
# # print(_1881.columns)
# # print(_1891.columns)

# # check if dateofbirth is in columns
# print("dateofbirth" in _1881.columns)
# print("dateofbirth" in _1891.columns)
# print("dateofbirth" in _1901.columns)
# print("dateofbirth" in _1911.columns)

True
True
True
False


### Phonetic

In [7]:
# %%time
# # 
# from recordlinkage.preprocessing import clean, phonetic
# # get name 
# # pname: Possibly "Personal Name" or "Given Name", referring to the individual's first name.
# # oname: Toname could potentially stand for "Other Name," which might include middle names, initials, or other identifiers. 
# # It could also be a field where various types of data have been entered, possibly due to data collection methods or data quality issues.
# # sname: This is generally understood to mean "Surname" or "Family Name", which is the individual's last name.

# name_cols = ['pname', 'sname']
# # age_cols = ['uk1861a_age']

# def gt_interation(df, cols):
#     # clean
#     for col in cols:
#         df[col] = clean(df[col], lowercase=False, remove_brackets=False)

#     # phonetic
#     for col in cols:
#         df[f'{col}_soundex'] = phonetic(df[col], method = 'soundex')
#     for col in cols:
#         df[f'{col}_metaphone'] = phonetic(df[col], method = 'metaphone')
        
#     return df

# # _1881 = gt_interation(_1881, name_cols)
# _1881 = gt_interation(_1881, name_cols)
# _1891 = gt_interation(_1891, name_cols)
# # _1901 = gt_interation(_1901, name_cols)

CPU times: user 3min 49s, sys: 23 s, total: 4min 12s
Wall time: 4min 12s


In [8]:
# def convert_age_to_numeric(age):
#     if age == 'less than 1 year':
#         return 0
#     elif age == '1 year':
#         return 1
#     elif age == '2 years':
#         return 2
#     elif age == '100+':
#         return 100
#     elif age == 'not reported/missing' or age == 'unkown':
#         return np.nan
#     else:
#         try:
#           return int(age)
#         except ValueError:
#           return np.nan

# # Apply the function to the age column
# census = {'1881':_1881, '1991':_1891}
# # census = {'1891':_1891, '1901':_1901}
# for k,v in census.items():
#   # v['cap_pname'] = v.pname.str.strip().str[0]
#   # v['cap_sname'] = v.sname.str.strip().str[0]

#   # v['age_numeric'] = v[f'uk{k}a_cage'].apply(convert_age_to_numeric)
#   v['dateofbirth'] = -v[f'uk{k}a_age'] + int(k)

In [9]:
# _1891.to_parquet("../Census_samples/Whole_proc/Whole_1891")
# _1881.to_parquet("../Census_samples/Whole_proc/Whole_1881")

In [ ]:
# subSample1861[['recid_mother', 'recid_father', 'recid_spouse', 'pname_mother',
#        'sname_mother', 'subdist_birth_ID_mother', 'age_mother', 'pname_father',
#        'sname_father', 'subdist_birth_ID_father', 'age_father', 'pname_spouse',
#        'sname_spouse', 'subdist_birth_ID_spouse', 'age_spouse', 'pname_sibl1',
#        'age_sibl1', 'pname_sibl2', 'age_sibl2', 'pname_sibl3', 'age_sibl3',
#        'pname_sibl4', 'age_sibl4', 'pname_sibl5', 'age_sibl5', 'pname_sibl6',
#        'age_sibl6', 'pname_sibl7', 'age_sibl7', 'pname_sibl8', 'age_sibl8',]]

### Feature engineering family ties

In [4]:
from dask.distributed import LocalCluster
client = LocalCluster(n_workers=32)

In [9]:
# _1881 = dd.read_parquet('../Census_samples/Whole_proc/Whole_1881')
# _1891 = dd.read_parquet('../Census_samples/Whole_proc/Whole_1891')
# _1901 = dd.read_parquet('../Census_samples/Whole_proc/Whole_1901') # print("dateofbirth" in _1901.columns) # True
_1911 = dd.read_parquet('../Census_samples/Whole_proc/Whole_1911') # print("dateofbirth" in _1911.columns) # True

In [17]:
@dask.delayed
def fam_ties_mom_pop_sp_dask(df, year):
    # Create a dictionary for each feature
    # age_dict = df.set_index('recid')['age'].to_dict()

    pname_dict = df.set_index('recid')['pname'].to_dict()
    sname_dict = df.set_index('recid')['sname'].to_dict()
    subdist_birth_id_dict = df.set_index('recid')['subdist_birth_ID'].to_dict()
    conparid_birth_dict = df.set_index('recid')['CONPARID_birth'].to_dict()

    # Map each dictionary to the respective new columns for mom, pop, and sp
    # df['age_mom'] = df['recid_mom'].map(age_dict.get)
    df['pname_mom'] = df['recid_mom'].map(pname_dict.get)
    df['sname_mom'] = df['recid_mom'].map(sname_dict.get)
    df['subdist_birth_ID_mom'] = df['recid_mom'].map(subdist_birth_id_dict.get)
    df['CONPARID_birth_mom'] = df['recid_mom'].map(conparid_birth_dict.get)

    # df['age_pop'] = df['recid_pop'].map(age_dict.get)
    df['pname_pop'] = df['recid_pop'].map(pname_dict.get)
    df['sname_pop'] = df['recid_pop'].map(sname_dict.get)
    df['subdist_birth_ID_pop'] = df['recid_pop'].map(subdist_birth_id_dict.get)
    df['CONPARID_birth_pop'] = df['recid_pop'].map(conparid_birth_dict.get)

    # df['age_sp'] = df['recid_sp'].map(age_dict.get)
    df['pname_sp'] = df['recid_sp'].map(pname_dict.get)
    df['sname_sp'] = df['recid_sp'].map(sname_dict.get)
    df['subdist_birth_ID_sp'] = df['recid_sp'].map(subdist_birth_id_dict.get)
    df['CONPARID_birth_sp'] = df['recid_sp'].map(conparid_birth_dict.get)

    df.to_parquet(f"../Census_samples/Whole_proc/Whole_{year}_mom_pop_sp")

# fam_ties_mom_pop_sp_dask(_1881, "1881").compute()
# fam_ties_mom_pop_sp_dask(_1891, "1891").compute()
# fam_ties_mom_pop_sp_dask(_1901, "1901").compute()
# fam_ties_mom_pop_sp_dask(_1911, "1911").compute()

In [26]:
import gc
gc.collect()
# _1881 = dd.read_parquet('../Census_samples/Whole_proc/Whole_1881_mom_pop_sp').compute()
# _1891 = dd.read_parquet('../Census_samples/Whole_proc/Whole_1891_mom_pop_sp').compute()
# _1901 = dd.read_parquet('../Census_samples/Whole_proc/Whole_1901_mom_pop_sp').compute()
_1911 = dd.read_parquet('../Census_samples/Whole_proc/Whole_1911_mom_pop_sp').compute()

In [27]:
def phonetic_iteration(df, cols):
    # clean
    for col in cols:
        df[col] = clean(df[col], lowercase=False, remove_brackets=False)

    # phonetic
    for col in cols:
        df[f'{col}_soundex'] = phonetic(df[col], method = 'soundex')
    for col in cols:
        df[f'{col}_metaphone'] = phonetic(df[col], method = 'metaphone')
        
    return df

# _1881 = gt_interation(_1881, name_cols)
# _1881 = gt_interation(_1881, name_cols)
# _1881 = phonetic_iteration(_1881, ['pname_mom', 'sname_mom', 'pname_pop', 'sname_pop',
#                'pname_sp', 'sname_sp'])
# _1891 = phonetic_iteration(_1891, ['pname_mom', 'sname_mom', 'pname_pop', 'sname_pop',
#                'pname_sp', 'sname_sp'])
# _1901 = phonetic_iteration(_1901, ['pname_mom', 'sname_mom', 'pname_pop', 'sname_pop',
#                 'pname_sp', 'sname_sp'])
_1911 = phonetic_iteration(_1911, ['pname_mom', 'sname_mom', 'pname_pop', 'sname_pop',
                'pname_sp', 'sname_sp'])

In [31]:
_1911.pname_pop_soundex.isna().sum() == _1911.pname_pop.isna().sum()
_1911.pname_pop_metaphone.isna().sum() == _1911.pname_pop.isna().sum()

True

In [30]:
# _1881.to_parquet(f"../Census_samples/Whole_proc/Whole_{1881}_mom_pop_sp")
# _1891.to_parquet(f"../Census_samples/Whole_proc/Whole_{1891}_mom_pop_sp")
# _1901.to_parquet(f"../Census_samples/Whole_proc/Whole_{1901}_mom_pop_sp")
_1911.to_parquet(f"../Census_samples/Whole_proc/Whole_{1911}_mom_pop_sp")

In [ ]:
# %%time
# # Assuming 'subSample1851' has 'recid', 'age', 'recid_mother', 'recid_father' columns
# # and 'pname_dict' and 'age_dict' are dictionaries mapping 'recid' to 'pname' and 'age' respectively.

# # Helper function to create sibling features based on a groupby object
# def create_sibling_features(df, parent_col):
#     # Group by the parent column and list siblings, excluding the same row (sibling)
#     sibling_info = df.groupby(parent_col)['recid'].apply(lambda x: x.tolist()).to_dict()

#     # Create a DataFrame from the dictionary
#     siblings_df = pd.DataFrame(list(sibling_info.items()), columns=[parent_col, 'siblings'])
#     siblings_df = siblings_df.explode('siblings')
#     siblings_df = siblings_df[siblings_df[parent_col] != siblings_df['siblings']]

#     # Merge back with the original DataFrame to get the age of each sibling
#     siblings_df = siblings_df.merge(df[['recid', 'age']], left_on='siblings', right_on='recid', suffixes=('', '_sib'))

#     # Sort by age within each family
#     # siblings_df.sort_values(by=[parent_col, 'age_sib'], inplace=True)

#     # Drop unnecessary columns
#     siblings_df.drop(columns=['recid'], inplace=True)

#     return siblings_df

# # Create sibling features for both mother and father
# maternal_siblings_df = create_sibling_features(subSample1851, 'recid_mother')
# paternal_siblings_df = create_sibling_features(subSample1851, 'recid_father')

# # Function to score and sort siblings based on relatedness
# def score_and_sort_siblings(maternal_df, paternal_df):
#     # Merge maternal and paternal siblings on 'siblings' (the recid of the sibling)
#     full_siblings_df = maternal_df.merge(paternal_df, on='siblings', suffixes=('_mat', '_pat'))
#     full_siblings = full_siblings_df['siblings'].tolist()

#     # Get half siblings from maternal and paternal, excluding full siblings
#     half_siblings = set(maternal_df['siblings']).union(paternal_df['siblings']) - set(full_siblings)
    
#     return full_siblings + list(half_siblings)

# # Now, we can iterate over the unique recids and create a sorted list of siblings
# combined_sorted_siblings = {}
# for recid in subSample1851['recid'].unique():
#     maternal = maternal_siblings_df[maternal_siblings_df['recid_mother'] == recid]
#     paternal = paternal_siblings_df[paternal_siblings_df['recid_father'] == recid]
#     combined_sorted_siblings[recid] = score_and_sort_siblings(maternal, paternal)

# # Map the personal name and age of siblings to the main DataFrame
# max_siblings = 8
# # for i in range(1, max_siblings + 1):
# #     subSample1851[f'pname_sibl{i}'] = subSample1851['recid'].map(lambda x: pname_dict.get(combined_sorted_siblings.get(x, [None] * max_siblings)[i-1]))
# #     subSample1851[f'age_sibl{i}'] = subSample1851['recid'].map(lambda x: age_dict.get(combined_sorted_siblings.get(x, [None] * max_siblings)[i-1]))
# for i in range(1, max_siblings + 1):
#     subSample1851[f'pname_sibl{i}'] = subSample1851['recid'].map(
#         lambda x: pname_dict.get(
#             combined_sorted_siblings.get(x, [None] * max_siblings)[i-1]
#         ) if i <= len(combined_sorted_siblings.get(x, [])) else None
#     )
#     subSample1851[f'age_sibl{i}'] = subSample1851['recid'].map(
#         lambda x: age_dict.get(
#             combined_sorted_siblings.get(x, [None] * max_siblings)[i-1]
#         ) if i <= len(combined_sorted_siblings.get(x, [])) else None
#     )

